## Auto Grade in PyTorch

In [417]:
import torch

In [418]:
# find differntiation of x^2 

def dy_dx(x):
    return 2*x

dy_dx(6)

12

In [419]:
import math
# complication inc y = x^2 , z = sin(y) now write a program 
# chain rule of diff 

def dz_dx(x):
    return 2*x * math.cos(x**2)

dz_dx(3)

-5.466781571308061

## Use autograde for upper equation

eg 1

In [420]:
x = torch.tensor(3.0,requires_grad=True) # require grade tell pytorch we use differentiation

y = x**2
print(x)
print(y) 

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)


In [421]:
y.backward() # find derivative of y
x.grad # use to show derivative (dy/dx)


tensor(6.)

eg 2

In [422]:
# y = x^2 and z = sin(y) find dz/dx 

x = torch.tensor(3.0,requires_grad=True)
y = x**2
z = torch.sin(y)

print(x)
print(y)
print(z) # it give value of z not diff of z

z.backward() # it give derivative
x.grad # must grad to starting one (dz/dx)

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)
tensor(0.4121, grad_fn=<SinBackward0>)


tensor(-5.4668)

### Make a type of NN 

### x -> w -> (Sigmoid(b)) -> y-Pred --> Loss

find dL/dw  // dL/db 

### 1. Linear Transformation : 

z = w.x + b

### 2. Activation(sigmoid Function):

y-Pred = sigma(z) = 1 /1 + e^-z

### 3. Loss Function (Binary Cross Entropy Loss)

L = -[y target.ln(y-pred) + (1 - y target).ln(1-ypred)]

---
### To Find

### x --> w --> (Sigmoid(b)) --> y-Pred --> Loss

1. dL/dw = dL/dy-Pred * d y-Pred/dz * dz/dw 

2. dL/db = dL/dy-Pred * d y-Pred/dz * dz/db

3. yp = y-Pred

4. dL/dyp = yp - y/yp (1-yp)  ,   dyp/dz = yp(1-yp)  ,  dz/dw = x  ,  dz/db = 1

5. dL\dw = (yp - y) * x   ,   dL/db = (yp - y)*1


## without auto grade how much it difficult to calculate it

In [423]:
# work on data a student got 6.7 cgpa and not got placement 
#analysy and predict

x = torch.tensor(6.7) #input feature
y = torch.tensor(0.0) # True label (library)

w = torch.tensor(1.0) #weight ( random value in w and b )
b = torch.tensor(0.0) #Bias

In [424]:
# Binary cross Entrophy Loss for scalar
def binary_cross_entropy_loss(prediction,target):
    epsilon = 1e-8 #to prevent log
    prediction = torch.clamp(prediction,epsilon,1-epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction)) 

In [425]:
# Forward pass
z = w * x + b # weight sum(linear part)
y_pred = torch.sigmoid(z) # predicted probability  ( sigmoid on z)

# compute binary cross entropy loss
loss = binary_cross_entropy_loss(y_pred,y)
loss  # ans show loss

tensor(6.7012)

In [426]:
# Derivatives : ( backpropogation)
# 1. dL/d(y_pred) : Loss with repect to prediction (y_pred)

dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

#2. dy_pred/dz : prediction (y_pred) with respect to z (sigmoid derivatives)
dy_pred_dz = y_pred * (1 - y_pred)

#3. dz/dw and dz/db: z with respect to w and b
dz_dw = x
dz_db = 1 # dz/db = 1 (bias contribution directly to z)

dl_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dl_db = dloss_dy_pred * dy_pred_dz * dz_db

In [427]:
print(f"Manual gradiant of loss w.r.t weight (dw) : {dl_dw}")
print(f"Manual gradiant of loss w.r.t bias (db) : {dl_db}")

Manual gradiant of loss w.r.t weight (dw) : 6.691762447357178
Manual gradiant of loss w.r.t bias (db) : 0.998770534992218


## With use of Autograde

In [428]:
x = torch.tensor(6.7) #( req_grad = True) not put bcz we are not calculate wrt to x or y
y = torch.tensor(0.0)

In [429]:
w = torch.tensor(1.0,requires_grad=True)
b = torch.tensor(0.0,requires_grad=True)

print(w)
print(b)

tensor(1., requires_grad=True)
tensor(0., requires_grad=True)


In [430]:
z = w * x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [431]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [432]:
# Calculate loss 
loss = binary_cross_entropy_loss(y_pred,y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [433]:
# Derivative ( this step will be easy to find derivative use backward)
# it is single NN it is very use full in big NN
loss.backward()
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


## Vector input tensor 

In [434]:
x = torch.tensor([1.0,2.0,3.0], requires_grad=True)
x

tensor([1., 2., 3.], requires_grad=True)

In [435]:
y = (x**2).mean() # y = f(x1,x2,x3)
y  #(1^2 + 2^2 + 3^2)/3

tensor(4.6667, grad_fn=<MeanBackward0>)

In [436]:
y.backward()
x.grad

# y = (x1^2 + x2^2 + x3^2)/3

# dy/dx1 = 2 x1/3 = 2/3 = 0.66
# dy/dx2 = 2 x2/3 = 4/3 = 1.33
# dy/dx3 = 2 x3/3 = 2 = 2

tensor([0.6667, 1.3333, 2.0000])

## Clearing gradian

In [437]:
# when we call grad multiple time cause problem
x = torch.tensor(2.0,requires_grad=True)
x

tensor(2., requires_grad=True)

In [438]:
# forward pass ( moving forward x -> y)
y = x**2
y

tensor(4., grad_fn=<PowBackward0>)

In [439]:
# backward pass ( x <- y)
y.backward()


In [440]:
x.grad  # if we run forward pass and backward pass 2nd time then ans change 4 -> 8


tensor(4.)

In [441]:
# Zero grad ( reset and fix the prob )
x.grad.zero_()  #  work as inplace =True

tensor(0.)

# Stop gradian tracking

In [442]:
y = x**2
y

tensor(4., grad_fn=<PowBackward0>)

In [443]:
y.backward()

In [444]:
x.grad
x

tensor(2., requires_grad=True)

In [445]:
#if we train NN in that case we off bakward tracking as req only forward tracking

#opt1 = required_grad_(False)
#opt2 = detach()
#opt3 = torch.no_grad()

### opt-1 (required_grad_(False))

In [446]:
x.requires_grad_(False)

tensor(2.)

In [447]:
#(requires_grad=True) is not showing after 2 bcz we off bakward tracking
x

tensor(2.)

In [448]:
# we cannot call backward on y as req grad is not there and stop 
y = x**2
y
#y.backward()

tensor(4.)

### opt2 = detach()


In [449]:
x = torch.tensor(2.0,requires_grad=True)

In [450]:
# make new tensor with function of x
# z is exact copy of x but gradian tracking is not ON

z = x.detach()
z

tensor(2.)

In [451]:
y = x ** 2
y # y has gradian function

tensor(4., grad_fn=<PowBackward0>)

In [452]:
y1 = z ** 2
y1  # y1 dont have grad funcion cannot do y1.backward()

tensor(4.)

### opt3 = torch.no_grad()

In [453]:
x = torch.tensor(2.0,requires_grad=True)
x

tensor(2., requires_grad=True)

In [454]:
with torch.no_grad():
    y = x ** 2
    
# if we remove with statement then tracking will be there

In [455]:
y # gradian is off so cannot do backward tracking

#y.backward()

tensor(4.)

 It depend on y as if x is req grad True then it do derivative , we have make it False or make a copy of x  in upper 2 cases while in 3 chase  with use of with torch.no_grad() it customize it
 
 --------------------------